####BSAN 6070 Machine Learning CA02 Spam Filtering Emails using Naive Bayes Classification
####**06 Feb 2022**

#**Mfolozi Dlamini**

# 1. Importing the necessary **modules** and **packages**

In [1]:
#Import modules/packages
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from google.colab import drive

#2. Link Working Directories



In [2]:
#mounting google drive to link the working directories
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
#link the working directories
test = '/content/drive/My Drive/Colab_Notebooks/test-mails'
train = '/content/drive/My Drive/Colab_Notebooks/train-mails'

#3. Define a function with **3000** most common words from all emails

####The purpose of this function is to build a dictionary of the most common 3000 words from all email content. First it adds all words and symbols in the dictionary. It then removes all non-alpha-numeric characters and any single character alpha-numeric characters. The dictionary is then shrunk keep only the most common 3000 words in the dictionary. The function returns the Dictionary.


In [4]:
def make_dictionary(root_dir):
  all_words_list = []

  #listing files from email folder(root_dir)]
  emails_list = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for email in emails_list:
    with open(email) as eml:
        for line in eml:
  #Splitting of the lines into individual words
          indiv_words = line.split()
  #Appending the individual words to a list
          all_words_list += indiv_words
  #Count the number of words within the list
    dictn = Counter(all_words_list)
  #Converting from count back to the list
    list_to_eliminate = list(dictn)

    for l in list_to_eliminate:
      if l.isalpha() == False:
  #Deletion of non-alphanumeric characters
        del dictn[l]
      elif len(l) == 1:
  #Deletion of single-lettered words like 'a'
        del dictn[l]
  #Creation of the final dictionary with 3000 most common words from email
    dictn = dictn.most_common(3000)
    return dictn
    

#4. Defining a function which extracts feature columns and populates their values


####This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [5]:
def features_extracted (email_dir):
#List of all files from folder
  email = [os.path.join(email_dir,doc) for doc in os.listdir(email_dir)]
#Making a list to store 3000 most common words and the number of files
  features_matrix = np.zeros((len(email),3000))
#Making an empty set of lables for the length of the number of files respectively
  labels_train = np.zeros(len(email))
  counts = 1;
  fileID = 0;
#For loop via listed files
  for e in email:
    with open(e) as il:
#Enumerating of the looped file
      for i, line in enumerate(il):
        if i ==2:
#Splitting of lines into individual words
          word = line.split()
          for w in word:
#Base wordID            
            wordID = 0
            for il, d in enumerate(dictn):
              if d[0] == w:
#Matching the looped word with the word from the dictionary
                wordID = il
#Counting the number of occurances in case of a match
                features_matrix[fileID,wordID] = word.count(word)
#Base EmailID (not being spam)
      labels_train[fileID] = 0;
      filepathTokens = e.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
#In case name of the file indicates training data to be a spam, the email gets reallocated
        labels_train[fileID] = 1;
#In case the file is marked as a spam, the variable wordcount is changed. This is to insinuate the occurance of the word in spam file.
        counts = counts + 1
#Creating new fileID(EmailID) for the following email
      fileID = fileID + 1
  return features_matrix, labels_train             

#5. Call the working directories again (as above)

In [6]:
#link the working directories
test = '/content/drive/My Drive/Colab_Notebooks/test-mails'
train = '/content/drive/My Drive/Colab_Notebooks/train-mails'

#6. Accuracy scores for predicting labels.

In [7]:
#Base dictonary for most common words utilizing the training set
dictn = make_dictionary(train)

#Extraction of the ID and the respective counts from the two sets (test, training)
print ("Reading and Processing Emails from Train and Test folders")
features_matrix, labels_train = features_extracted(train)
test_features_matrix, labels_test = features_extracted(test)

#Gauss Naive Bayes Model (This model accounts the words to be normally distributed and are independent of each other)
gnb_model = GaussianNB()

#Check the features
print ("Training Model using Gaussian Naive Bayes Algorithm .....")
gnb_model.fit(features_matrix, labels_train)
print ("Training Complete")
print ("Testing Trained model in order to predict Test Data labels")
labels_predicted = gnb_model.predict(test_features_matrix)
print ("Completed Test Data classification.... printing now Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(labels_test, labels_predicted))

Reading and Processing Emails from Train and Test folders
Training Model using Gaussian Naive Bayes Algorithm .....
Training Complete
Testing Trained model in order to predict Test Data labels
Completed Test Data classification.... printing now Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.5


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:489: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
